In [1]:
import os
from pathlib import Path
from string import Template

import yaml

from cognite.client import CogniteClient, global_config

file_path = Path("cognite-sdk-config.yaml")

# Read in yaml file and substitute environment variables in the file string
env_sub_template = Template(file_path.read_text())
file_env_parsed = env_sub_template.substitute(dict(os.environ))

# Load yaml file string into a dictionary to parse global and client configurations
cognite_config = yaml.safe_load(file_env_parsed)

# If you want to set a global configuration it must be done before creating the client
global_config.apply_settings(cognite_config["global"])
client = CogniteClient.load(cognite_config["client"])

In [95]:
from cognite.client.data_classes import TimeSeriesList
import pandas as pd

def dump_timeseries_to_csv(timeseries_list: TimeSeriesList, filename: str) -> None:
    # Convert TimeSeriesList to a pandas DataFrame
    df = timeseries_list.to_pandas(
    # Expand the metadata column (assumed to be a dictionary) into separate columns
    metadata_df = df['metadata'].apply(pd.Series)

    # Combine the original DataFrame with the expanded metadata
    df = pd.concat([df.drop(columns=['metadata']), metadata_df], axis=1)

     # Remove all columns that start with 'context'
    df = df.loc[:, ~df.columns.str.startswith('context')]

    columns_to_drop = ['id', 'asset_id', 'created_time', 'last_updated_time', 'is_string', 'security_categories']

    # Drop the specified columns
    df = df.drop(columns=columns_to_drop)

    print("Columns after drop:", df.columns.tolist())


    # Write the resulting DataFrame to a CSV file
    df.to_csv(filename, index=False)



ts : TimeSeriesList = client.time_series.list(limit=-1, external_id_prefix="pi:")
dump_timeseries_to_csv(ts, "pi.csv")

Columns after drop: ['external_id', 'name', 'is_step', 'description', 'unit', 'compdev', 'location5', 'pointtype', 'convers', 'descriptor', 'digitalset', 'zero', 'filtercode', 'compdevpercent', 'compressing', 'tag', 'srcptid', 'displaydigits', 'recno', 'excdev', 'userreal2', 'userreal1', 'totalcode', 'ptclassname', 'typicalvalue', 'pointid', 'pointsource', 'location1', 'location2', 'location3', 'location4', 'shutdown', 'compmin', 'ptclassrev', 'scan', 'userint2', 'userint1', 'excdevpercent', 'archiving', 'instrumenttag', 'compmax', 'ptclassid', 'squareroot', 'sourcetag', 'excmin', 'exdesc', 'future', 'excmax', 'step', 'engunits', 'span', '_replicatedSource', '_replicatedTime', '_replicatedInternalId']
